In [6]:
import sys
# !{sys.executable} -m pip install qiskit_aer
!{sys.executable} -m pip install qiskit[visualization]


zsh:1: no matches found: qiskit[visualization]


In [5]:
import math
from qiskit_aer import Aer
from qiskit_aer.utils import transpile_noise_model
from qiskit import transpile, QuantumCircuit
from qiskit.circuit.library import GroverOperator, MCMT, ZGate
from qiskit.visualization import plot_histogram

ImportError: cannot import name 'ProviderV1' from 'qiskit.providers' (unknown location)

In [ ]:
from qiskit_aer.noise import NoiseModel
from qiskit_aer.noise import depolarizing_error

In [ ]:
# To run on Aer simulator:
backend = Aer.get_backend('qasm_simulator')

def grover_oracle(marked_states):
    """Build a Grover oracle for multiple marked states

    Here we assume all input marked states have the same number of bits

    Parameters:
        marked_states (str or list): Marked states of oracle

    Returns:
        QuantumCircuit: Quantum circuit representing Grover oracle
    """
    if not isinstance(marked_states, list):
        marked_states = [marked_states]
    # Compute the number of qubits in circuit
    num_qubits = len(marked_states[0])

    qc = QuantumCircuit(num_qubits)
    # Mark each target state in the input list
    for target in marked_states:
        # Flip target bit-string to match Qiskit bit-ordering
        rev_target = target[::-1]
        # Find the indices of all the '0' elements in bit-string
        zero_inds = [ind for ind in range(num_qubits) if rev_target.startswith("0", ind)]
        # Add a multi-controlled Z-gate with pre- and post-applied X-gates (open-controls)
        # where the target bit-string has a '0' entry
        qc.x(zero_inds)
        qc.compose(MCMT(ZGate(), num_qubits - 1, 1), inplace=True)
        qc.x(zero_inds)
    return qc

In [ ]:
marked_states = ["011", "100"]

oracle = grover_oracle(marked_states)

grover_op = GroverOperator(oracle)

optimal_num_iterations = math.floor(
    math.pi / (4 * math.asin(math.sqrt(len(marked_states) / 2**grover_op.num_qubits)))
)

In [ ]:
qc = QuantumCircuit(grover_op.num_qubits)
# Create even superposition of all basis states
qc.h(range(grover_op.num_qubits))
# Apply Grover operator the optimal number of times
qc.compose(grover_op.power(optimal_num_iterations), inplace=True)
# Measure all qubits
qc.measure_all()

In [ ]:
# Defining noise models for gates above
def get_noise_model():
    # Error probs
    error_probs = {
        "reset": 0.03, 
        "x": 0.03,      
        "cx": 0.05      
    }

    # Create a depolarizing noise model
    noise_model = NoiseModel()
    for gate_name, prob in error_probs.items():
        noise_model.add_quantum_error(depolarizing_error(prob, 1))
    return noise_model

In [ ]:
try:
    # Noise model
    noise_model = get_noise_model()

    circ_ = transpile(qc, backend)
    circ_ = transpile_noise_model(circ_, noise_model) # Apply noise model
    job = backend.run(circ_)
    result = job.result()
    counts = result.get_counts()

    plot_histogram(counts)
    print(counts)
except Exception as e:
    print(e)
    print("Please use the Aer provider to run the simulation")

name 'get_noise_model' is not defined
Please use the Aer provider to run the simulation


## Factorization

Now we're going to compare quantum and classical factorization.

For our classical factorization, we're going to use sympy. 

In [ ]:
from sympy import factorint

def classical_factorization(n):
    return factorint(n)

print("Classical Factorization of 123456789: ", classical_factorization(123456789))


Classical Factorization of 123456789:  {3: 2, 3607: 1, 3803: 1}


In [ ]:
!{sys.executable} -m pip install matplotlib

  Using cached contourpy-1.2.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 4.5 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.5-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.4 kB)
  Using cached pillow-10.3.0-cp311-cp311-macosx_11_0_arm64.whl.metadata (9.2 kB)
  Using cached pyparsing-3.1.2-py3-none-any.whl.metadata (5.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 13.4 MB/s eta 0:00:00a 0:00:01
Using cached contourpy-1.2.1-cp311-cp311-macosx_11_0_arm64.whl (245 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.9 MB/s eta 0:00:0000:0100:01
Using cached kiwisolver-1.4.5-cp311-cp311-macosx_11_0_arm64.whl (66 kB)
Using cached pillow-10.3.0-cp311-cp311-macosx_11_0_arm64.whl (3.4 MB)
Using cached pyparsing-3.1.2-py3-none-any.whl (103 kB)


In [8]:
!{sys.executable} -m pip show qiskit

Name: qiskit
Version: 1.1.0
Summary: An open-source SDK for working with quantum computers at the level of extended quantum circuits, operators, and primitives.
Home-page: 
Author: 
Author-email: Qiskit Development Team <qiskit@us.ibm.com>
License: Apache 2.0
Location: /Users/devg/Downloads/class_notes/.venv/lib/python3.11/site-packages
Requires: dill, numpy, python-dateutil, rustworkx, scipy, stevedore, symengine, sympy, typing-extensions
Required-by: qiskit-aer, qiskit-algorithms


In [7]:
# imports for RSA
from numpy import gcd
from numpy.random import seed, randint
# imports for Shor
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
from qiskit.circuit.library import QFT

ImportError: cannot import name 'QuantumCircuit' from 'qiskit' (unknown location)

In [ ]:
seed(1)

N = 100
# a = randint(2, N) # 1 < a < N
a = 13

if gcd(a, N) == 1: # a shares no factors
    print(f"{1} < {a} < {N}, {1 < a < N}")
else: # a shares a factor
    P = gcd(a, N)
    Q = N // gcd(a, N)
    print(f"P = {P}\nQ = {Q}\n\n",
          f"{P} x {Q} = {N}, {P * Q == N}\n")
    print("You got lucky! You can skip to the Decypting 213 section, I guess. 😂")

1 < 13 < 100, True


In [ ]:
def initialize_qubits(qc, n, m):
    qc.h(range(n)) # apply hadamard gates
    qc.x(n+m-1) # set qubit to 1

In [ ]:
print(f"Which in your case is\n\tU(x) = a^x mod {N}")
def a_x_mod15(a, x):
    if a not in [2,7,8,11,13]:
        raise ValueError("'a' must be 2,7,8,11 or 13")
    U = QuantumCircuit(4)        
    for iteration in range(x):
        if a in [2,13]:
            U.swap(0,1)
            U.swap(1,2)
            U.swap(2,3)
        if a in [7,8]:
            U.swap(2,3)
            U.swap(1,2)
            U.swap(0,1)
        if a == 11:
            U.swap(1,3)
            U.swap(0,2)
        if a in [7,11,13]:
            for q in range(4):
                U.x(q)
    U = U.to_gate()
    U.name = f"U({x})"
    c_U = U.control()
    return c_U
def modular_exponentiation(qc, n, m, a):
    for x in range(n):
        exponent = 2**x
        qc.append(a_x_mod15(a, exponent), 
                     [x] + list(range(n, n+m)))

Which in your case is
	U(x) = a^x mod 100


In [ ]:
def apply_iqft(qc, measurement_qubits):
    qc.append(QFT(len(measurement_qubits),
                             do_swaps=False).inverse(),
                         measurement_qubits)

In [ ]:
def measure(qc, n):
    qc.measure(n, n)

In [ ]:
def period_finder(n, m, a):
    
    # set up quantum circuit
    qc = QuantumCircuit(n+m, n)
    
    # initialize the qubits
    initialize_qubits(qc, n, m)
    qc.barrier()

    # apply modular exponentiation
    modular_exponentiation(qc, n, m, a)
    qc.barrier()

    # apply inverse QFT
    apply_iqft(qc, range(n))
    qc.barrier()

    # measure the n measurement qubits
    measure(qc, range(n))
    
    return qc

In [ ]:
n = 4; m = 4

qc = period_finder(n, m, a)
qc.draw(output='mpl')

MissingOptionalLibraryError: "The 'matplotlib' library is required to use 'MatplotlibDrawer'. You can install it with 'pip install matplotlib'."